## Can we reach the stars with fusion power?

Several improvements over earlier versions:

* Have discarded the idea of an ion rocket fueled with D2O in favor of a more conventional fusion rocket fueled with D2.  Calculations indicate that this is feasible, and the travel times are much less.  This also eliminates the need to generate TW of electrical power.  We still need ~1GW of electrical power for the colony, but this will be much easier.
* Have refined the rocket equations to allow for an initial velocity.  Travel times are reduced significantly if we can tow the craft up to a decent initial velocity before leaving the solar system.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from constants import CGS, MKS
from IPython.display import Image

## Merlin rocket benchmark.
This is from the Wikipedia entry on the Falcon Merlin engine:
As configured for use on Falcon 1 vehicles, the Merlin 1C had a sea level thrust of 350 kN (78,000 lbf), a vacuum thrust of 400 kN (90,000 lbf) and a vacuum specific impulse of 304 s (2.98 km/s). In this configuration, the engine consumed 140 kg (300 lb) of propellant per second.

In addition, we need the fact that kerosene has a heat of combustion of 46.2 MJ/kg.  Kerosene is approximately $C_{14}H_{30}$  Complete combustion of one molecule of kerosene would require 43 O atoms.

The tabulated gas temperature in the combustion chamber is about 3400C.

In [ ]:
# So the heat of combustion including the oxidizer is:
Ef = 46.2E6 *(14.0*12.0 + 30.0) / ( 14.0 * 12.0 + 30.0 + 43.0 * 16.0) 
print("Ef = %.3g MJ/kg of fuel"%(Ef/1.0E6))
# Power used in fuel consumption:
Mf = 140.0 # kg of fuel per second
Pf = Mf * Ef # Energy produced by fuel combustion per second
Veff = 3.0E3 # Effective exhaust velocity
Pe = Mf * Veff * Veff / 2.0 # Kinetic energy of exhaust per second
eta = Pe / Pf # efficiency
print("Power of combustion = %.3g J/s, Exhaust power = %.3g J/s, Efficency = %.1f%%"%(Pf,Pe,eta*100.0))
# How does the exhaust velocity compare to the thermal velocity of the gas molecules?
# Exhaust is mainly CO2 (MW 44)  and H2O (MW 18).
T_f = 3400.0 + 273.0 # T in degrees K
V_f = np.sqrt(2.0 * T_f * MKS.kb / ((18.0+44.0)/2 * MKS.mp))
print("Thermal velocity of gas = %.4g m/s compared to tabulated exhaust velocity of %.4g m/s"%(V_f,Veff))

## D-D Fusion.

First some basic quantities:

* Assume D2 fuel, DD fusion with efficiency $\eta$ at turning fusion power into exhaust power.  
* DD fusion energy = $\rm E_{DD}$ = 10 MeV per DD pair.
* $\rm V_{eff} = \sqrt{2 \eta E_{DD} / m_{DD}}$

In [ ]:
# How does this compare to a chemical rocket?
# The numbers are much larger, but appear feasible.
eta = 0.40
mDD = 4.0
EDD = 1.0E7 * MKS.eV
Veff = np.sqrt(2 * eta * EDD / (mDD * MKS.mp))
print("Veff = %.4g m/s = %.1f%% of c"%(Veff,Veff/MKS.c*100.0))
T_p = 1.0E6 # Fusion temperature of 1 MeV, near peak of D-D cross section
V_He = np.sqrt(2.0 * T_p * MKS.eV / (4.0 * MKS.mp))
print("At T_plasma = %.2g eV = %.4g K"%(T_p, T_p*MKS.eV/MKS.kb))
print("Thermal velocity of gas = %.1f%% of c compared to exhaust velocity of %.1f%% of c"%(V_He/MKS.c*100.0,Veff/MKS.c*100.0))

## Mass definitions

Let's define the following quantities:
* m0 - Mass of craft with no fuel
* mt - turnover mass
* mi - initial mass - craft plus fuel
* mdot - rate of fuel usage
* D = total distance; T = total trip time
* v0 = Initial velocity - assuming a tug to get started

  Now let's calculate these quantities, assuming the rocket equation:
$\rm v_f = v_0 + v_{eff} \log{\frac{m_i}{m_f}}$
* Since the velocity at turnover is the same accelerating and declerating, we must have:
* $\rm \log(\frac{m_t}{m_0}) = \log(\frac{m_i}{m_t}) + v_0$
* $\rm m_t = \sqrt{m_0 m_i} \exp(\frac{v_0}{2 v_{eff}})= \sqrt{m_0 (m_0 + mdot T)} \exp(\frac{v_0}{2 v_{eff}})$

In [ ]:
# I've refined these now to allow for an initial velocity.
# Velocities are in m/s, distance in LY, and times in years
# Masses in kg, and mdot in kg/s

def Velocity(V0, Veff, mf, mi, mdot, sign):
    # From the rocket equation
    # sign = 1.0: accel; sign = -1.0: decel    
    return V0 + sign * Veff * np.log(mi / mf)

def Distance(V0, Veff, mf, mi, mdot, sign):
    # From integrating the rocket equation
    # sign = 1.0: accel; sign = -1.0: decel
    t = abs((mi - mf) / mdot)
    x0 = V0 * t
    x1 = Veff * t
    x2 = Veff * (mi / mdot) * (1.0 - mdot * t / mi) * np.log(1.0 - mdot * t / mi)
    return (x0 + sign * x1 + sign * x2) / MKS.ly

def TotalDistance(V0, Veff, m0, mdot, T):
    # Total distance traveled given m0, mdot, V0, and time T
    mi = m0 + mdot * T * MKS.yr
    mt = np.sqrt(m0 * (m0 + mdot * T * MKS.yr)) * np.exp( V0 / (2.0 * Veff))
    Tt = (mi - mt) / mdot / MKS.yr
    Vt = Velocity(V0, Veff, mt, mi, mdot, 1.0) # V at turnover
    d_accel = Distance(V0, Veff, mt, mi, mdot, 1.0)
    d_decel = Distance(Vt, Veff, m0, mt, mdot, -1.0)
    return d_accel + d_decel

def TotalTime(V0, Veff, m0, mdot, D, Tguess):
    # Invert TotalDistance(T) using Newton's method
    # Gives total time to travel a distance D given m0, mdot, and V0
    d = TotalDistance(V0, Veff, m0, mdot, Tguess)
    T = Tguess
    error = D - d
    count = 0
    while error > 1.0E-9 and count < 20:
        d = TotalDistance(V0, Veff, m0, mdot, T)
        dp = TotalDistance(V0, Veff, m0, mdot, T * 1.01)
        dDdT = (dp - d) / (T * .01)
        T = T - (d - D) / dDdT
        d = TotalDistance(V0, Veff, m0, mdot, T)
        error = abs(D - d)
        count += 1
    if count > 15:
        print("Failed to converge")
        return 0.0
    else:
        return T


In [ ]:
nsteps = 500
m0 = 2.0E8 # Empty mass in kg. About 2 aircraft carriers
mdot = 0.20 # Fuel usage in kg/s
D = 4.3 # Travel distance in LY (Alpha Centauri)
plt.figure(figsize = (16,8))
for v0 in [0.0, 2.0]:
    print("Initial velocity = %.1f%% of c"%v0)
    V0 = v0 / 100.0 * MKS.c
    T = TotalTime(V0, Veff, m0, mdot, D, 100)
    print("Total time = %.1f years"%T)
    mi = m0 + mdot * TotalTime(V0, Veff, m0, mdot, D, 100) * MKS.yr
    print("Initial mass = %.2g kg = %.1f times m0"%(mi,mi/m0))
    mt = np.sqrt(m0 * (m0 + T * mdot * MKS.yr)) * np.exp( V0 / (2.0 * Veff))
    print("Turnover mass = %.2g kg = %.1f times m0"%(mt,mt/m0))
    Tt = (mi - mt) / mdot / MKS.yr
    print("Turnover time = %.1f years"%Tt)
    Vp = Velocity(V0, Veff, mt, mi, mdot, 1.0) / MKS.c * 100.0
    print("Peak velocity = %.1f%% of c"%Vp)
    Thrust = Veff * mdot
    Ai = Thrust / mi / 9.8
    Af = Thrust / m0 / 9.8
    print("Thrust = %.3g Nt"%Thrust)
    print("Initial acceleration = %.3g g, Final acceleration = %.3g g"%(Ai,Af))
    print("Fusion power = %.1f TW"%(mdot * EDD / (mDD * MKS.mp) / 1.0E12))
    print("Exhaust power = %.1f TW"%(mdot * Veff**2 / 2.0E12))

    times = np.linspace(0, T, nsteps)
    dt = T / float(nsteps - 1)
    speeds = []
    distances = [0.0]
    Vt = Velocity(V0, Veff, mt, mi, mdot, 1.0) # At turnover
    Dt = Distance(V0, Veff, mt, mi, mdot, 1.0) # At turnover
    for t in times:
        m = mi - mdot * t * MKS.yr
        if t < Tt: # Accel
            speeds.append(Velocity(V0, Veff, m, mi, mdot, 1.0) / MKS.c)
            distances.append(Distance(V0, Veff, m, mi, mdot, 1.0))
        else: # Decel
            speeds.append(Velocity(Vt, Veff, m, mt, mdot, -1.0) / MKS.c)
            distances.append(Dt + Distance(Vt, Veff, m, mt, mdot, -1.0))
    distances.remove(distances[-1])
    plt.subplot(1,2,1)
    plt.plot(times, speeds, label = 'V0 = %.1f%% of c'%v0)
    plt.text(Tt*0.95, Vp/100.0*1.05, "Turnover", fontsize=12)
    plt.subplot(1,2,2)
    plt.plot(times, distances, label = 'V0 = %.1f%% of c'%v0)
    print("")
plt.subplot(1,2,1)
plt.title("Speed vs time")
plt.ylim(0, 0.08)
plt.xlabel("Time(yr)")
plt.ylabel("Speed (fraction of c)")
plt.legend()
plt.subplot(1,2,2)
plt.title("Distance vs time")
plt.ylim(0, 5.0)
plt.xlabel("Time(yr)")
plt.ylabel("Distance (ly)")
plt.legend()
plt.show()

In [ ]:
# What about going further?
# There are more than 20 stars within 12 LY
m0 = 2.0E8
mdot = 0.20
V0 = 0.04 * MKS.c
D = 12.0
T = TotalTime(V0, Veff, m0, mdot, D, 100)
print("Total time = %.1f years"%T)
mi = m0 + mdot * TotalTime(V0, Veff, m0, mdot, D, 100) * MKS.yr
print("Initial mass = %.2g kg = %.1f times m0"%(mi,mi/m0))
mt = np.sqrt(m0 * (m0 + T * mdot * MKS.yr)) * np.exp( V0 / (2.0 * Veff))
print("Turnover mass = %.2g kg = %.1f times m0"%(mt,mt/m0))
Tt = (mi - mt) / mdot / MKS.yr
print("Turnover time = %.1f years"%Tt)
Vp = Velocity(V0, Veff, mt, mi, mdot, 1.0) / MKS.c * 100.0
print("Peak velocity = %.1f%% of c"%Vp)

In [ ]:
# How long does it take to tow it up to initial speed?
# Assume the tow distance equal to the diameter of Jupiter's orbit.
# So it would take about 1.5 years to get to 2% of c, but this saves
# more than 60 years of travel time.

As = []
Vs = []
Ts = []
for i in range(100):
    a = 0.0001 * float(i+1)
    As.append(a)
    d = 10.0 * MKS.AU # Diameter of Jupiter's orbit
    v = np.sqrt(2*a*d)
    Vs.append(v / MKS.c *100.0)
    Ts.append(v / a / MKS.yr)

plt.figure(figsize = (16,8))
plt.subplot(1,2,1)
plt.plot(As, Vs) 
plt.xlabel("Acceleration ( m/s)")
plt.ylabel("Velocity (% of c)")
plt.subplot(1,2,2)
plt.plot(As, Ts) 
plt.xlabel("Acceleration ( m/s)")
plt.ylabel("Time required (yr)")
plt.show()

In [ ]:
# How large does the plasma volume need to be?
# Assume the following values:

sigmaV = 2.0E-22 # m^3/s
n = 1.0E20 # Density in m^-3
Rho_f = EDD * n * n * sigmaV
print("Power density = %.3g W/m^3"%Rho_f)
P_f = mdot * EDD / (mDD * MKS.mp)
Vol = P_f / Rho_f
Radius = (3.0 / (4.0 * np.pi) * Vol)**(1.0/3.0)
print("Fusion volume = %.3g m^3. Radius of %.1f m"%(Vol, Radius))

In [ ]:
# What is the thickness of the fuel shell?
# Density of solid D2 is about 160 kg/m^3

rho_fuel = 160.0 # kg/m^3
m_fuel = mi - m0 # kg
V_shell = m_fuel / rho_fuel # m^3
A_shell = 2.0 * np.pi * 100.0 * 600.0 + 2.0 * np.pi * 100.0**2 # m^2
T_shell = V_shell / A_shell
print("Fuel shell thickness is about %.1f m"%T_shell)
Image(filename='/Users/cslage/Personal/Starfarer/Vessel.png')

In [ ]:
# What would be the rotational speed?  Assume 0.8g
acc = 0.8 * 9.8 # m/s^2
omega = np.sqrt(acc / 100.0)
RPM = 2.0 * np.pi * 60.0 * omega
print("Rotational speed = %.1f RPM"%RPM)